In [1]:
# Using test data to build/prototype-out a ML model
    # Further restructuring data for ML input layers
        # Variable isolation and selection refinement
    # Reshaping, encoding, normalization
    # Input, layer, and output specification
    # Force keras/tensorflow to use GPU backend

In [2]:
# Imports
import pandas as pd
import numpy as np
import json
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline

# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# ML Imports 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU
from keras.models import load_model
from keras import backend as K

# Preprocessing Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from keras.utils import to_categorical 

# Import to check check for GPU availability for tensorflow backend
from tensorflow.python.client import device_lib

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
# Mongodb connection for live feed of data into model
    # Implementation after verification of test data -> model prototype input layer working

In [3]:
# Verify GPU availability for tensorflow backend
print(device_lib.list_local_devices())
print("==============================================")
print(K.tensorflow_backend._get_available_gpus())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5974383275331631333
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9222031934
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1832398150829761340
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
['/job:localhost/replica:0/task:0/device:GPU:0']


In [4]:
# Read in test data for L2 orderbook state (bids + asks)
    # Read in test data for subsquent L2 orderbook update states (L2 updates to bid + asks)
snapshot_asks_df = pd.read_csv("test_data/snapshot_asks.csv")
snapshot_bids_df = pd.read_csv("test_data/snapshot_bids.csv")
l2update_df = pd.read_csv("test_data/l2update.csv", dtype ={'changes':object})
request_log_df= pd.read_csv("test_data/request_log.csv")

In [12]:
print(request_log_df.head())
print("===============================================")
print(snapshot_asks_df.head())
print(snapshot_bids_df.head())
print("===============================================")
print(l2update_df.head())
print(l2update_df.tail())

      Unnamed: 0                       iso         epoch
0    request end  2018-04-17T01:05:03.469Z  1.523927e+09
1  request start  2018-04-17T00:55:03.354Z  1.523927e+09
     price       size  side
0  8042.91  10.656147  sell
1  8042.92   0.001181  sell
2  8042.93   0.001305  sell
3  8042.94   0.001181  sell
4  8042.95   1.265966  sell
     price      size side
0  8042.90  9.118009  buy
1  8042.83  0.002487  buy
2  8042.50  0.420000  buy
3  8042.00  0.001000  buy
4  8041.33  0.020000  buy
  side    price   size                      time
0  buy  8041.33  0.000  2018-04-17T00:55:04.358Z
1  buy  8041.43  0.020  2018-04-17T00:55:04.375Z
2  buy  7940.12  0.000  2018-04-17T00:55:04.395Z
3  buy  8039.00  0.001  2018-04-17T00:55:04.412Z
4  buy  7972.56  0.000  2018-04-17T00:55:04.413Z
       side    price      size                      time
18088  sell  8078.07  0.000000  2018-04-17T01:05:03.247Z
18089   buy  8039.05  0.000000  2018-04-17T01:05:03.330Z
18090  sell  8040.52  0.005328  2018-04-

In [13]:
# Bid= buy
# Ask = sell

#L2 snapshot structure
    # [side,price,size]
    # 'side' added as part of structure for classification
    
#l2 updates structure
    # [side, price, size, time]

# Note on GDAX API about l2update structure:
    # size of "0" indicates the price level can be removed